In [2]:
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel
from typing import List, Optional

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import cv2
import numpy as np

import torch
import base64
import random
from matplotlib import pyplot as plt

C:\Users\halil\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
xalil_model = torch.hub.load("ultralytics/yolov5","custom",path="/Users/halil/Desktop/gity/map92.pt",force_reload=False)

Using cache found in C:\Users\halil/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-26 Python-3.8.10 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
img_batch = cv2.imread("images/logo_det.png")
rgb = cv2.cvtColor(img_batch, cv2.COLOR_BGR2RGB)

#results = xalil_model(img_batch, size = 640)

In [ ]:
cv2.imshow("asdas",pred)
cv2.waitKey(0)

In [6]:
def results_to_json(results, model):
    ''' Converts yolo model output to json (list of list of dicts)'''
    return [
                [
                    {
                    "class": int(pred[5]),
                    "class_name": model.model.names[int(pred[5])],
                    "bbox": [int(x) for x in pred[:4].tolist()], #convert bbox results to int from float
                    "confidence": float(pred[4]),
                    }
                for pred in result
                ]
            for result in results.xyxy
            ]

In [7]:
def plot_one_box(x, im, color=(128, 128, 128), label=None, line_thickness=3):
    # Directly copied from: https://github.com/ultralytics/yolov5/blob/cd540d8625bba8a05329ede3522046ee53eb349d/utils/plots.py
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def base64EncodeImage(img):
    ''' Takes an input image and returns a base64 encoded string representation of that image (jpg format)'''
    _, im_arr = cv2.imencode('.jpg', img)
    im_b64 = base64.b64encode(im_arr.tobytes()).decode('utf-8')
    return im_b64
colors = [tuple([random.randint(0, 255) for _ in range(3)]) for _ in range(100)] 

In [16]:
xalil_model.conf = 0.5
xalil_model.classes = None


predictions = xalil_model(rgb,size=640)
json = results_to_json(predictions,xalil_model)

In [26]:
for  results in predictions.xyxy:
    for result in results:
        print(result[4])


tensor(0.97071)
tensor(0.96864)
tensor(0.96447)
tensor(0.96063)
tensor(0.96057)
tensor(0.95985)
tensor(0.95580)
tensor(0.94003)
tensor(0.90477)
tensor(0.86653)
tensor(0.83399)
tensor(0.77290)
tensor(0.76663)
tensor(0.69803)
tensor(0.69729)
tensor(0.67905)
tensor(0.64835)
tensor(0.63204)
tensor(0.52999)
tensor(0.51663)


In [24]:
for i in json[0]:
    print(i)

{'class': 0, 'class_name': 'apple', 'bbox': [827, 397, 901, 485], 'confidence': 0.9707061052322388}
{'class': 0, 'class_name': 'apple', 'bbox': [373, 355, 513, 532], 'confidence': 0.9686378836631775}
{'class': 0, 'class_name': 'apple', 'bbox': [291, 610, 410, 754], 'confidence': 0.9644722938537598}
{'class': 0, 'class_name': 'apple', 'bbox': [548, 606, 674, 769], 'confidence': 0.9606329798698425}
{'class': 0, 'class_name': 'apple', 'bbox': [20, 354, 166, 535], 'confidence': 0.9605681896209717}
{'class': 0, 'class_name': 'apple', 'bbox': [623, 415, 680, 479], 'confidence': 0.959848940372467}
{'class': 0, 'class_name': 'apple', 'bbox': [230, 403, 300, 489], 'confidence': 0.9558010101318359}
{'class': 0, 'class_name': 'apple', 'bbox': [1250, 651, 1301, 714], 'confidence': 0.9400272369384766}
{'class': 3, 'class_name': 'google', 'bbox': [503, 842, 549, 891], 'confidence': 0.904768705368042}
{'class': 0, 'class_name': 'apple', 'bbox': [803, 292, 830, 321], 'confidence': 0.8665332198143005}


In [33]:
img_str_list = []
#plot bboxes on the image
for img, bbox_list in zip(img_batch, json):
    for bbox in bbox_list:
        label = f'{bbox["class_name"]} {bbox["confidence"]:.2f}'
        plot_one_box(bbox['bbox'], img, label=label, 
                color=colors[int(bbox['class'])], line_thickness=3)

    img_str_list.append(base64EncodeImage(img))


In [34]:
img_str_list

['/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/wAALCAWgAAMBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APxr1ID+0J8k/wCtbsfWm6pkajOP+mp659fal1EKL+YH/nof50mpZN/MQCP3h4zTtTAOoTZJ/wBaegPrSaoD/aM44/1p6k+vtRqQX7fNn/noelGogm/mI4/eHjNO1IA6hMTnPmtnH1pNTB/tGcHH+tbOfrSakF+3zA/89D0+tGpZ+3zYBH7w8A0upjOozkk5809AfWk1UEalOP8Apoeuf6UmqKv9ozAj/loaXUs/2hNgEfvDxmnaoB/aU+Sf9aegPrSasD/ac/T/AFrdSfX2o1QAajMD18w9KNTz/aM2Dj94eM07VQDqU5Of9aen1pNXBGqXAOP9a3X60aqqDUpw3XzTnn3o1XP9pT44/eHjNLqyg6nOTn/WtnH1pNXBGqXA4/1rdfrRqwUalOD1800mq5OpTkHH7w8Zp2rgHVLgkn/Wt0+tJrGf7UuOn+ubr9aNWC/2nPn/AJ6n+dGrAnU5yDj94eM0urrnVLg5/wCWrdvek1kEarcDj/XN1+tLq4QapcAg581u/vRq5P8Aak5Bx+9PGaXWADqtwTnPnNnH1pus7hq1wO